# Practica cinco

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica se usará la librería de numpy, pandas, matplotlib, y sklearn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Ejercicio 1

## Ejercicio 2